Глава 6. Чтение и запись данных, форматы файлов

In [1]:
import pandas as pd
import numpy as np

# 6.1 Чтение и запись данных в текстовом формате

- Доступ к данным – обязательный первый шаг применения большинства описанных в книге инструментов.

- Индексирование: какие столбцы рассматривать как индекс возвращаемого DataFrame и откуда брать имена столбцов: из файла, от пользователя
или вообще ниоткуда.
- Выведение типа и преобразование данных: включает определенные пользователем преобразования значений и список маркеров отсутствующих
данных.
- Разбор даты и времени: включает средства комбинирования, в том числе
сбор данных о дате и времени из нескольких исходных столбцов в один
результирующий.
- Итерирование: поддержка обхода очень больших файлов.
- Проблемы грязных данных: пропуск заголовка или концевика, комментариев и другие мелочи, например обработка числовых данных, в которых
тройки разрядов разделены запятыми.

##### Таблица 6.1. Функции чтения в pandas
- read_csv Загружает данные с разделителями из файла, URL-адреса или похожего на файл
объекта. По умолчанию разделителем является запятая
- read_table Загружает данные с разделителями из файла, URL-адреса или похожего на файл
объекта. По умолчанию разделителем является символ табуляции ('\t')
- read_fwf Читает данные в формате с фиксированной шириной столбцов (без разделителей)
- read_clipboard Вариант read_table, который читает данные из буфера обмена. Полезно
для преобразования в таблицу данных на веб-странице
- read_excel Читает табличные данные из файлов Excel в формате XLS или XLSX
- read_hdf Читает HDF5-файлы, записанные pandas
- read_html Читает все таблицы, обнаруженные в HTML-документе
- read_json Читает данные из строки в формате JSON (JavaScript Object Notation)
- read_msgpack Читает данные pandas, представленные в двоичном формате MessagePack
- read_pickle Читает произвольный объект, хранящийся в формате Python pickle
- read_sas Читает набор данных в одном из пользовательских форматов хранения системы SAS
- read_sql Читает результаты SQL-запроса (применяя пакет SQLAlchemy) в объект pandas
DataFrame
- read_stata Читает набор данных из файла в формате Stata
- read_feather Читает данные из двоичного файла в формате Feather

In [2]:
# !type печатает содержимое файла на экране без какого-либо форматирования.
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
df = pd.read_csv('ex1.csv')
# или pd.read_table('ex1.csv', sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# Рассмотрим этот же файл без заголовка
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [5]:
pd.read_csv('ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [6]:
# Заголовок по умолчанию
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# Сами задаем заголовок
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# Допустим, мы хотим, чтобы столбец message стал индексом возвращаемого
# объекта DataFrame. Этого можно добиться, задав аргумент index_col,
# в котором указать, что индексом будет столбец с номером 4 или
# с именем 'message':
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'],
            index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
!type csv_mindex.csv
parsed = pd.read_csv('csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [10]:
# Иногда в таблице нет фиксированного разделителя, а для разделения
# полей используются пробелы или еще какой-то символ. В таком случае
# можно передать функции read_table регулярное выражение вместо разделителя.
# Рассмотрим такой текстовый файл:
list(open('ex3.csv'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491']

In [11]:
# В данном случае поля разделены переменным числом пробелов и, хотя
# можно было бы переформатировать данные вручную, проще передать функции
# read_table регулярное выражение \s+ в качестве разделителя:
result = pd.read_table('ex3.csv', sep='\s+')
result
# Поскольку имен столбцов на одно меньше, чем строк, read_table делает
# вывод, что в данном частном случае первый столбец должен быть индексом DataFrame.

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [12]:
# Параметр skiprows позволяет пропустить строки файла:
!type ex4.csv
pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
#  Отсутствующие значения обычно либо вообще опущены (пустые строки),
# либо представлены специальными маркерами. По умолчанию в pandas
# используется набор общеупотребительных маркеров: NA, -1.#IND и NULL:
!type ex5.csv
result = pd.read_csv('ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [14]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [15]:
# Параметр na_values может принимать список или множество строк,
# рассматриваемых как маркеры отсутствующих значений:
sentinels = {'message': ['foo'], 'something': ['two']}
pd.read_csv('ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


##### Таблица 6.2. Часто используемые аргументы функций read_csv и read_table
- $path$ Строка, обозначающая путь в файловой системе, URL-адрес или похожий на файл
объект
- $sep$ или
$delimiter$
Последовательность символов или регулярное выражение, служащее для разделения
полей в строке
- $header$ Номер строки, содержащей имена столбцов. По умолчанию равен 0 (первая строка).
Если строки-заголовка нет, должен быть равен None
- index_col - Номера или имена столбцов, трактуемых как индекс строк в результирующем
объекте. Может быть задан один номер (имя) или список номеров (имен),
определяющий иерархический индекс
- names - Список имен столбцов результирующего объекта; задается, если header=None
- skiprows - Количество игнорируемых начальных строк или список номеров игнорируемых строк
(нумерация начинается с 0)
- na_values - Последовательность значений, интерпретируемых как маркеры отсутствующих данных
- comment - Один или несколько символов, начинающих комментарий, который продолжается
до конца строки
- parse_dates - Пытаться разобрать данные как дату и время; по умолчанию False. Если равен True,
то производится попытка разобрать все столбцы. Можно также задать список столбцов,
которые следует объединить перед разбором (если, например, время и даты заданы
в разных столбцах)
- keep_date_col - В случае когда для разбора данных столбцы объединяются, следует ли отбрасывать
объединенные столбцы. По умолчанию True
converters Словарь, содержащий отображение номеров или имен столбцов на функции.
Например, {'foo': f} означает, что нужно применить функцию f ко всем значениям
в столбце foo
- dayfirst - При разборе потенциально неоднозначных дат предполагать международный формат
(т. е. 7/6/2012 означает «7 июня 2012 года»). По умолчанию False
- date_parser - Функция, применяемая для разбора дат
nrows Количество читаемых строк от начала файла
- iterator - Возвращает объект TextParser для чтения файла порциями
- chunksize - Размер порции при итерировании
- skip_footer - Сколько строк в конце файла игнорировать
- verbose - Печатать разного рода информацию о ходе разбора, например количество
отсутствующих значений, помещенных в нечисловые столбцы
- encoding - Кодировка текста в случае Unicode. Например, 'utf–8' означает, что текст представлен
в кодировке UTF-8
- squeeze - Если в результате разбора данных оказалось, что имеется только один столбец,
вернуть объект Series
- thousands - Разделитель тысяч, например ',' или '.'

## Чтение текстовых файлов порциями

In [16]:
# Прежде чем приступать к обработке большого файла, попросим pandas
# отображать меньше данных:
# P.S. теперь можно смотреть до 100 значений и не будут выдаваться три точки
pd.options.display.max_rows = 100

In [17]:
result = pd.read_csv('ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [18]:
# Для чтения файла порциями задайте с помощью
# параметра chunksize размер порции в строках:
pd.read_csv('ex6.csv', nrows=15)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [19]:
# Объект TextParser, возвращаемый функцией read_csv, позволяет читать файл
# порциями размера chunksize. Например, можно таким образом итеративно
# читать файл ex6.csv, агрегируя счетчики значений в столбце 'key':

# Для чтения файла порциями задаем с помощью
# параметра chunksize размер порции в строках:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

C:\Users\A4F7~1\AppData\Local\Temp/ipykernel_448/287897813.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [20]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
V    328.0
I    327.0
U    326.0
P    324.0
D    320.0
A    320.0
R    318.0
Y    314.0
G    308.0
S    308.0
N    306.0
W    305.0
T    304.0
B    302.0
Z    288.0
C    286.0
4    171.0
6    166.0
7    164.0
8    162.0
3    162.0
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
dtype: float64

## Вывод данных в текстовом формате

In [21]:
# Данные можно экспортировать в формате с разделителями. Рассмотрим
# одну из приведенных выше операций чтения CSV-файла:
data = pd.read_csv('ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
# С помощью метода to_csv объекта DataFrame мы можем вывести данные
# в файл через запятую:
data.to_csv('out.csv', sep='|')
!type out.csv

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [23]:
import sys
data.to_csv(sys.stdout, sep='|')
# или 
# data.to_csv('out.csv', sep='|')
# !type out.csv

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [24]:
# Отсутствующие значения представлены пустыми строками, но можно
# вместо этого указать какой-нибудь маркер:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [25]:
# Если не указано противное, выводятся метки строк и столбцов. Но и те,
# и другие можно подавить:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [26]:
# Можно также вывести лишь подмножество столбцов, задав их порядок:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


- Series

In [27]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [28]:
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [29]:
ts.to_csv('tseries.csv')
!type tseries.csv
# чет криво выводится результат

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## Обработка данных в формате с разделителями

In [30]:
!type ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [31]:
# Для любого файла с односимвольным разделителем можно воспользоваться
# стандартным модулем Python csv:
import csv
f = open('ex7.csv')
reader = csv.reader(f)

In [32]:
# Итерирование файла с помощью объекта reader дает кортежи значений
# в каждой строке после удаления кавычек:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [33]:
# Далее можно произвести любые манипуляции, необходимые для преобразования
# данных к нужному виду. Пойдем по шагам. Сначала прочитаем файл в
# список строк:
with open('ex7.csv') as f:
    lines = list(csv.reader(f))

lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [34]:
# Затем отделим строку-заголовок от остальных:
header, values = lines[0], lines[1:]

In [35]:
# Далее мы можем создать словарь столбцов, применив словарное включение
# и выражение zip(*values), которое транспонирует строки и столбцы:

data_dict = {h: v for h, v in zip(header, zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Для установления нового формата со $своим$ разделителем, соглашением об употреблении кавычек
и способе завершения строк необходимо определить простой подкласс класса
csv.Dialect:

In [36]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

f = open('ex7.csv')
reader = csv.reader(f, dialect=my_dialect)

for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


Для $записи$ файлов с разделителями (вручную) можно использовать метод csv.
writer. Он принимает объект, который представляет открытый, допускающий
запись файл и те же параметры диалекта и форматирования, что csv.reader:

In [37]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## Данные в формате JSON

Формат JSON (JavaScript Object Notation) стал очень популярен для обмена
данными по протоколу HTTP между веб-сервером и браузером или другим
клиентским приложением. Этот формат обладает куда большей гибкостью,
чем табличный текстовый формат типа CSV. Данные в формате JSON очень напоминают код на Python с тем отличием,
что отсутствующее значение обозначается null.

In [38]:
# Для преобразования JSON-строки в объект Python служит метод json.loads:

obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [39]:
# Напротив, метод json.dumps преобразует объект Python в формат JSON:

asjson = json.dumps(result)

asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [40]:
# Для удобства предлагается возможность передать список словарей 
# (которые раньше были объектами JSON) конструктору DataFrame и 
# выбрать подмножество полей данных:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


Функция pandas.read_json умеет автоматически преобразовывать наборы
данных определенного вида в формате JSON в объекты Series или DataFrame.
Подразумеваемые по умолчанию параметры pandas.read_json предполагают,
что каждый объект в JSON-массиве – это строка таблицы:
Например:

In [41]:
!type example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [42]:
data = pd.read_json('example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


## XML и HTML: разбор веб-страниц

pandas.read_html по умолчанию ищет и пытается разобрать все табличные данные внутри тегов table.
Результатом является список объектов DataFrame:

In [43]:
tables = pd.read_html('fdic_failed_bank_list.html')
len(tables)

1

Получили массив нескольких таблиц. Выведем первую таблицу:

In [44]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Посчитаем количество банкротств по годам:

In [45]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [46]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

### Разбор XML с помощью lxml.objectify

XML (расширяемый язык разметки) – еще один популярный формат представления структурированных данных, поддерживающий иерархически вложенные данные, снабженные метаданными.
- Форматы XML и HTML структурно похожи, но XML более общий.

Временные ряды с данными о работе автобусов и электричек. Для каждой автобусной и железнодорожной
компании существует свой файл (например, Performance_MNR.xml для компании
MetroNorth Railroad), содержащий данные за месяц в виде последовательности таких XML-записей:

Дальше не понял

# 6.2 Двоичные форматы данных

Дальше не понял

## Формат HDF5

- HDF5 – хорошо зарекомендовавший себя файловый формат для хранения
больших объемов научных данных в виде массивов. 
- HDF5-файл содержит внутри себя структуру узлов, напоминающую файловую
систему, которая позволяет хранить несколько наборов данных вместе с относящимися к ним метаданными.
- Для очень больших наборов данных, которые не помещаются в память, HDF5 – отличный выбор, потому что дает возможность эффективно
читать и записывать небольшие участки гораздо больших массивов.

Дальше пока мне не нужно

## Чтение файлов Microsoft Excel

In [47]:
# Чтение данных из рабочего листа в объект DataFrame.
# Прочтение первого листа:

# pd.read_excel('Abiturients.xlsx', 'Лист1')

In [48]:
# frame = pd.read_excel('ex1.xlsx', 'Sheet1')

Для записи данных pandas в файл формата Excel следует сначала создать
объект ExcelWriter, а затем записать в него данные, пользуясь методом to_excel объектов pandas:

In [49]:
# writer = pd.ExcelWriter('ex2.xlsx')
# frame.to_excel(writer, 'Sheet1')
# writer.save()

In [50]:
# Можно вместо этого передать путь к файлу методу to_excel, избежав тем
# самым создания ExcelWriter:

# frame.to_excel('ex2.xlsx')

# 6.3. Взаимодейтсиве с HTML и Web API

Многие сайты предоставляют открытый API для получения данных в формате JSON или каком-то другом. 

In [51]:
# Чтобы найти последние 30 заявок, касающихся pandas на GitHub,
# мы можем отправить с помощью библиотеки requests такой HTTP-запрос GET:

import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [52]:
# Метод json объекта Response возвращает словарь, содержащий JSON-данные,
# представленные в виде объектов Python:

data = resp.json()

data[0]['title']

'BUG/TST/DOC: added finalize to melt, GH28283'

In [53]:
# Каждый элемент в списке data – словарь, содержащий все данные на
# странице заявки в GitHub (кроме комментариев). Список data можно
# передать конструктору DataFrame и выделить интересующие нас поля:

issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues.head()

,number,title,labels,state
0,46648,"BUG/TST/DOC: added finalize to melt, GH28283",[],open
1,46647,CI/DOC: Unpin jinja2,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
2,46646,BUG: Make categories setitem error more readable,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,46645,TST: Regression test added for Timedelta,[],open
4,46644,BUG attempt to fix GH46575,[],open


# 6.4. Взаимодейтсиве с базами данных

В корпоративных системах большая часть данных хранится в реляционных базах данных на основе SQL (например, SQL Server, PostgreSQL и MySQL). 

In [54]:
# В качестве примера возьмем базу данных SQLite
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect('mydata.sqlite')
# con.execute(query)
# con.commit()

In [55]:
# Затем вставим несколько строк в таблицу:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [56]:
# Большинство драйверов SQL, имеющихся в Python (PyODBC, psycopg2,
# MySQLdb, pymssql и т. д.), при выборе данных из таблицы 
# возвращают список кортежей:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [57]:
# Имена столбцов содержатся в атрибуте курсора description:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [58]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5
9,Atlanta,Georgia,1.25,6


Такое переформатирование не хочется выполнять при каждом запросе
к базе данных. Проект SQLAlchemy (www.sqlalchemy.org) – популярная библиотека на Python, абстрагирующая многие различия между базами данных
SQL. В pandas имеется функция read_sql, которая позволяет без труда читать
данные из соединения, открытого SQLAlchemy. В примере ниже мы подключаемся к той же самой базе SQLite с помощью SQLAlchemy и читаем данные
из ранее созданной таблицы:

In [59]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5
9,Atlanta,Georgia,1.25,6
